In [216]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


# Create Graph with a state of all messages

In [217]:
from langgraph.graph import add_messages, StateGraph
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    event_type: str
    location: str
    labels: str
    
graph_builder = StateGraph(State)

# Connection to Couchbase's vector store

In [218]:
from langchain_openai import OpenAIEmbeddings
import os
from couchbase.cluster import Cluster
from couchbase.options import ClusterOptions
from couchbase.auth import PasswordAuthenticator
from langchain_couchbase import CouchbaseVectorStore

COUCHBASE_CONNECTION_STRING = os.environ["COUCHBASE_CONNECTION_STRING"]
COUCH_USER = os.environ["COUCHBASE_USER"]
COUCH_PASS = os.environ["COUCHBASE_PASS"]
BUCKET_NAME = os.environ["COUCHBASE_BUCKET"]
SCOPE_NAME = os.environ["COUCHBASE_SCOPE"]
COLLECTION_NAME = os.environ["COUCHBASE_COLLECTION"]
SEARCH_INDEX_NAME = os.environ["COUCHBASE_SEARCH_INDEX"]

auth = PasswordAuthenticator(COUCH_USER, COUCH_PASS)
options = ClusterOptions(auth)
cluster = Cluster(COUCHBASE_CONNECTION_STRING, options)
embedding = OpenAIEmbeddings()
embedding.model = "text-embedding-3-small"

vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embedding,
    index_name=SEARCH_INDEX_NAME,
)

# Create Graph instance

In [219]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")
template = """You are a helpful bot that serves the purpose of finding events for artists looking for venues in the USA. If you cannot answer based on the context provided, respond with a generic
    answer. Answer the question as truthfully as possible using the context below: {context}
Please also format the result in Markdown format.
            Question: {question}"""

prompt = ChatPromptTemplate.from_template(template)

generation_chain = prompt | llm | StrOutputParser()


def chatbot(state: State):
    response = generation_chain.invoke({"context": state['messages'], "question": f"We are a {state['event_type']} amateur group looking for {state['labels']} festivals in {state['location']}, can you please recommend some for us to reach out to?"})
    state['messages'].append(response)
    return state

def search_couchbase(state: State):
    query = f"""
    Geo Info: {state['location']}
    Category: {state['event_type']}
    """
    retriever = vector_store.as_retriever()
    results = retriever.invoke(query)
    print(f"RAG Search results: {results}")
    # state['messages'].append(HumanMessage(f"Search results: {results}"))
    # print(f"number of messages in state: {len(state['messages'])}")
    # print(f"Message number 1 of state: {state['messages'][0]}")
    # print(f"Message number 2 of state: {state['messages'][1]}")
    return state

graph_builder.add_node("vector_search", search_couchbase)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("vector_search")
graph_builder.add_edge("vector_search", "chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)


# Interaction

In [222]:
from random import randint
from IPython.core.display import Markdown

session_id = randint(1, 10000)
config = {"configurable": {"thread_id": session_id}}

input_location = "Florida, USA"
input_category = "Music"
input_labels = "music"
for event in graph.stream({"event_type": input_category, "location": input_location, "labels": input_labels}, config):
    for value in event.values():
        if len(value['messages']) > 0:
            display(Markdown(value['messages'][-1]))

RAG Search results: []


Sure! Here are some music festivals in Florida, USA that you can reach out to:

1. **Ultra Music Festival** - Miami
2. **SunFest** - West Palm Beach
3. **Gasparilla Music Festival** - Tampa
4. **Okeechobee Music & Arts Festival** - Okeechobee
5. **Tortuga Music Festival** - Fort Lauderdale

I hope this helps!